# 07 · Prediction intervals — conformal via residual calibration

In [ ]:
import numpy as np, pandas as pd, torch, glob, os
from src.dataset import split_dataframe, build_matrices, load_scaler
from src.model import MLP
from src.calibration import residual_calibration_intervals

df = pd.read_csv('../data/processed/dataset.csv')
feature_cols = [c for c in df.columns if c.startswith('x')]
target_cols = ['H2_ppm','Propane_ppm']
train_df, val_df, test_df = split_dataframe(df, split_col='split')
X_val, y_val = build_matrices(val_df, feature_cols, target_cols)
X_test, y_test = build_matrices(test_df, feature_cols, target_cols)
scaler = load_scaler('../models/scaler.joblib')
X_val = scaler.transform(X_val).astype('float32')
X_test = scaler.transform(X_test).astype('float32')

ckpt_path = sorted(glob.glob('../models/best_model_*.pt'))[0]
ckpt = torch.load(ckpt_path, map_location='cpu')
model = MLP(ckpt['in_dim'], ckpt['hidden'], ckpt['out_dim'])
model.load_state_dict(ckpt['state_dict']); model.eval()

with torch.no_grad():
    ypv = model(torch.from_numpy(X_val)).numpy()
    ypt = model(torch.from_numpy(X_test)).numpy()

lower, upper = residual_calibration_intervals(y_val, ypv, ypt, alpha=0.1)
coverage = ((y_test >= lower) & (y_test <= upper)).mean(axis=0)
print('Per-target coverage (~0.9 expected):', coverage)